In [1]:
import pandas as pd
import numpy as np

In [2]:
def get_data(filename='kalendar.xlsx'):
    df = pd.read_excel(filename)
    df['Результат оцінювання'] = df['Результат оцінювання'].str.strip()
    df['Рішення (посилання)'] = df['Рішення (посилання)'].str.strip()
#     df = df.drop_duplicates(['Рішення (посилання)', 'Область'], keep='last')
    df['Результат оцінювання'] = df['Результат оцінювання'].str.strip()
    df['Дата кваліфоцінювання'] = df['Дата кваліфоцінювання'].astype('datetime64[s]')
    df['Дата кваліфоцінювання'] = df['Дата кваліфоцінювання'].dt.floor('d')
    df['Порушення доброчесності'] = df['Порушення доброчесності'].fillna(0)
    df['Чи є профайл'] = df['Чи є профайл'].fillna(0)
    return df

def cleaning_results(df):
    df['Результат оцінювання'] = np.select(
    condlist=[ 
        df['Результат оцінювання'].str.contains('зупинено', na=False),
        df['Результат оцінювання'].str.contains('припинено', na=False),
        df['Результат оцінювання'].str.contains('перерв', na=False),
        df['Результат оцінювання'].str.contains('відкладен', na=False),
        df['Результат оцінювання'].str.contains('не відповідає', na=False),
        df['Результат оцінювання'].str.contains('відповідає займаній посаді|відповідає|подання про звільнення|призначення|переведення|рекомендовано', na=False),
        df['Результат оцінювання'].str.contains('знято', na=False),
        df['Результат оцінювання'].str.contains('http', na=False),
        df['Результат оцінювання'].str.contains("не з'явився", na=False),
        df['Результат оцінювання'].str.contains('відкладено', na=False),
        df['Результат оцінювання'].str.contains('відсторонений', na=False),
        df['Результат оцінювання'].str.contains('відмова', na=False),
    ],
    choicelist=['Зупинено', 'Припинено', 'Перерва', 'Відкладено', 'Не відповідає', 'Відповідає', 'Знято',
                'Звільнений', "Не з'явився",'Відкладено', 'Тимчасово відсторонений', 'Відмова від проходження'],
    default='Категорія відсутня')
    return df

def cleaning_oblast(df):
    df['Область'] = np.select(
    condlist=[ 
        df['Область'].str.contains('Київськ', na=False, case=False),
        df['Область'].str.contains('ки[їє]в', na=False, case=False),
        df['Область'].str.contains('харк', na=False, case=False),
        df['Область'].str.contains('львів', na=False, case=False),
        df['Область'].str.contains('дніпро', na=False, case=False),
        df['Область'].str.contains('закарп', na=False, case=False),
        df['Область'].str.contains('Хмельн', na=False, case=False),
        df['Область'].str.contains('одес', na=False, case=False),
        df['Область'].str.contains('полтав', na=False, case=False),
        df['Область'].str.contains('Житомир', na=False, case=False),
        df['Область'].str.contains('Волин', na=False, case=False),
        df['Область'].str.contains('рівн', na=False, case=False),
        df['Область'].str.contains('донець', na=False, case=False),
        df['Область'].str.contains('микола', na=False, case=False),
        df['Область'].str.contains('Черкас', na=False, case=False),
        df['Область'].str.contains('кіровог', na=False, case=False),
        df['Область'].str.contains('запорі', na=False, case=False),
        df['Область'].str.contains('вінни', na=False, case=False),
        df['Область'].str.contains('луган', na=False, case=False),
        df['Область'].str.contains('терноп', na=False, case=False),
        df['Область'].str.contains('черніг', na=False, case=False),
        df['Область'].str.contains('чернів', na=False, case=False),
        df['Область'].str.contains('івано', na=False, case=False),
        df['Область'].str.contains('херсон', na=False, case=False),
        df['Область'].str.contains('сум', na=False, case=False),
        df['Область'].str.contains('крим', na=False, case=False),
    ],
    choicelist=['Київcька область', 'м. Київ', 'Харківська область', 'Львівська область',
               'Дніпропетровська область', 'Закарпатська область', 'Хмельницька область',
               'Одеська область', 'Полтавська область', 'Житомирська область',
               'Волинська область', 'Рівненська область', 'Донецька область',
               'Миколаївська область', 'Черкаська область', 'Кіровоградська область',
               'Запорізька область', 'Вінницька область', 'Луганська область',
               'Тернопільська область', 'Чернігівська область', 'Чернівецька область',
               'Івано-Франківська область', 'Херсонська область', 'Сумська область',
               'Кримська область'],
    default='Категорія відсутня')
    return df

def count_q(df):
    counts = df['Результат оцінювання'].value_counts().reset_index()
    counts.columns = ['Статус', 'Кількість']
    # counts = counts.loc[counts['Статус'].ne('Тимчасово відсторонений')]
    counts['perc'] = (counts['Кількість'].astype(int) / len(df)) * 100
    counts['perc'] = counts['perc'].round(2).astype(str) + '%'
    counts.columns = ['Статус', 'К-сть', 'К-сть, %']
    counts['К-сть недоброчесних'] = [len(df.loc[df['Порушення доброчесності'].eq(1) & df['Результат оцінювання'].eq(C)]) for C in counts['Статус'].values]
    counts['К-сть недоброчесних, % від категорії'] = (counts['К-сть недоброчесних'].astype(int) / counts['К-сть'].astype(int)) * 100
    counts['К-сть недоброчесних, % усіх'] = (counts['К-сть недоброчесних'].astype(int) / len(df)) * 100
    counts['К-сть недоброчесних, % від категорії'] = counts['К-сть недоброчесних, % від категорії'].round(2).astype(str) + '%'
    counts['К-сть недоброчесних, % усіх'] = counts['К-сть недоброчесних, % усіх'].round(2).astype(str) + '%'
    counts = counts.append(counts.sum(numeric_only=True), ignore_index=True)
    counts.iloc[-1, 0] = 'Загалом'
    counts.iloc[-1, np.r_[2, 4, 5]] = '100%'
    counts['К-сть'] = counts['К-сть'].astype(int)
    counts['К-сть недоброчесних'] = counts['К-сть недоброчесних'].astype(int)
    return counts

def group_data(df):
    g = df.groupby(['Область', 'Результат оцінювання', 'Порушення доброчесності']).size().unstack(fill_value=0)
    grouped = g.reset_index()
    grouped.columns = ['Область', 'Результат оцінювання', 'Доброчесний', 'Недоброчесний']
    grouped['Загалом'] = grouped['Доброчесний'].astype(int) + grouped['Недоброчесний'].astype(int)
    return grouped

def recategorize(df):
    new_categories = {
        "Відповідає": ['Відповідає'], 
        "Не відповідає": ['Не відповідає'],
        "Інше": ['Звільнений', 'Знято', 'Перерва',
           'Відкладено', 'Зупинено', "Не з'явився", 'Відмова від проходження',
           'Припинено']}
    categories = {v: k for k,vv in new_categories.items() for v in vv}
    new_df = df.copy()
    new_df['Результат оцінювання'] = new_df['Результат оцінювання'].map(categories)
    return new_df 

In [3]:
df = get_data()
df = cleaning_results(df)
df = cleaning_oblast(df)
df = df.drop_duplicates(['Рішення (посилання)', 'Область'], keep='last')
counts = count_q(df)
counts

,Статус,К-сть,"К-сть, %",К-сть недоброчесних,"К-сть недоброчесних, % від категорії","К-сть недоброчесних, % усіх"
0,Відповідає,570,61.96%,52,9.12%,5.65%
1,Перерва,82,8.91%,9,10.98%,0.98%
2,Знято,81,8.8%,21,25.93%,2.28%
3,Відкладено,63,6.85%,23,36.51%,2.5%
4,Не з'явився,35,3.8%,18,51.43%,1.96%
5,Зупинено,27,2.93%,3,11.11%,0.33%
6,Відмова від проходження,24,2.61%,1,4.17%,0.11%
7,Звільнений,13,1.41%,2,15.38%,0.22%
8,Не відповідає,13,1.41%,6,46.15%,0.65%
9,Припинено,12,1.3%,0,0.0%,0.0%


In [4]:
group_data(df).head()

,Область,Результат оцінювання,Доброчесний,Недоброчесний,Загалом
0,Івано-Франківська область,Відкладено,1,0,1
1,Івано-Франківська область,Відповідає,13,0,13
2,Івано-Франківська область,Звільнений,1,0,1
3,Івано-Франківська область,Знято,3,0,3
4,Івано-Франківська область,Зупинено,3,0,3


In [5]:
recategorised_counts = recategorize(df)
count_q(recategorised_counts)

,Статус,К-сть,"К-сть, %",К-сть недоброчесних,"К-сть недоброчесних, % від категорії","К-сть недоброчесних, % усіх"
0,Відповідає,570,61.96%,52,9.12%,5.65%
1,Інше,337,36.63%,77,22.85%,8.37%
2,Не відповідає,13,1.41%,6,46.15%,0.65%
3,Загалом,920,100%,135,100%,100%


In [6]:
df.groupby(['Область', 'Результат оцінювання']).size().unstack(fill_value=0)

Результат оцінювання,Відкладено,Відмова від проходження,Відповідає,Звільнений,Знято,Зупинено,Не відповідає,Не з'явився,Перерва,Припинено
Область,,,,,,,,,,
Івано-Франківська область,1,0,13,1,3,3,0,0,1,0
Волинська область,1,0,16,0,2,1,1,0,3,0
Вінницька область,2,0,23,1,2,2,1,0,5,1
Дніпропетровська область,5,0,45,1,5,4,1,0,9,1
Донецька область,2,2,48,1,5,2,0,0,3,0
Житомирська область,2,0,25,0,0,1,0,0,4,2
Закарпатська область,2,0,9,0,0,2,1,0,3,0
Запорізька область,1,0,16,1,3,0,1,0,1,2
Київcька область,1,0,8,0,4,0,0,0,4,1


---

In [146]:
df['Дата кваліфоцінювання'] = df['Дата кваліфоцінювання'].astype('datetime64[s]')
df['Місяць'] = df['Дата кваліфоцінювання'].dt.month

In [148]:
# пікові місяці
df['Місяць'].value_counts()

7     237
6     236
5     204
4      75
3      61
8      23
2       2
10      1
Name: Місяць, dtype: int64